# OptiGuide Example




In [1]:
# Install Required Packages
%pip install optiguide

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import re
import requests  # for loading the example source code
import openai
import os

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

In [17]:
openai.api_key_path = "OAI_CONFIG_LIST.txt"

In [18]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            "gpt4",
            "gpt-4-32k",
            "gpt-4-32k-0314",
            "gpt-3.5-turbo",
            "gpt-3.5-turbo-16k",
            "gpt-3.5-turbo-0301",
            "chatgpt-35-turbo-0301",
            "gpt-35-turbo-v0301",
        }
    }
)


Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [19]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/yuqingsh/capstone-project-demo/master/farm_lotsizing.py"
response  = requests.get(code_url)
# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")
# code = open(code_url, "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

import time
from gurobipy import GRB, Model

# Example data

suppliers = ['supplier1', 'supplier2']
farms = ['farm1', 'farm2']
markets = ['market1', 'market2']

seed_supply_capacity = {'supplier1': 1000, 'supplier2': 1500}
.
.
.


# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)


In [20]:
# In-context learning examples.
example_qa = """
----------
Question: Why is it not recommended to rely on just one seed supplier for Farm 2?
Answer Code:
```python
z = model.addVars(suppliers, vtype=GRB.BINARY, name="z")
model.addConstr(sum(z[s] for s in suppliers) <= 1, "one_supplier_constraint")
for s in suppliers:
    model.addConstr(x[s, 'farm2'] <= seed_supply_capacity[s] * z[s], "capacity_constraint_farm2")
```



"""

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [21]:
%%capture
agent = OptiGuideAgent(name="OptiGuide Coffee Example",
                  source_code=code,
                   debug_times=1,
                  example_qa="",
                llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    })

user = UserProxyAgent("user", max_consecutive_auto_reply=0,
                         human_input_mode="NEVER", code_execution_config=False)


Now, let's create a user's question.

In [22]:
user.initiate_chat(agent, message="How can we adjust the model if we want to explore a scenario where the planting cost at Farm 1 increases by 10%?")


user (to OptiGuide Coffee Example):

How can we adjust the model if we want to explore a scenario where the planting cost at Farm 1 increases by 10%?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Increase the planting cost at farm1 by 10%
planting_cost['farm1'] = planting_cost['farm1'] * 1.10
```

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to safeguard):


--- Code ---
# Increase the planting cost at farm1 by 10%
planting_cost['farm1'] = planting_cost['farm1'] * 1.10

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Coffee Example):

SAFE

---------------------------------------------------------------------